# Internet and Websites Search using Bing API - Bing Chat Clone

이 노트북에서는, Langchain과 Azure Bing Search API 서비스를 모두 활용하여 웹 검색 기능으로 GPT Smart Search Engine을 부스팅할 수 있는 방법에 대해 알아보겠습니다.

앞서, 다른 노트북에서 설명했듯이 harnessing agent 와 tools는 효과적인 접근법입니다. 우리는 GPT-4 및 그 후속제품과 같은 OpenAI의 대규모 언어모델(LLM)을 활용하여 추론 및 연구를 대신하여 수행하는 것을 목표로 하고 있습니다.

스마트 검색 엔진에 인터넷 접속이 필요한 경우는 여러 가지가 있습니다. 예를 들어, **웹에서 이용 가능한 정보를 제공하여 답변을 풍부하게 하거나**, **사용자에게 최신 및 최근 정보를 제공하거나**, **특정 공개 웹사이트에서 정보를 찾는 등**의 경우에도 검색 결과를 기반으로 엔진의 응답을 요구할 수 있습니다.

이 노트를 진행하면 **Bing Search API를 사용하여 Web Search Agent를 생성하는 방법**과 이러한 도구가 챗봇을 강화하는 방법 등 Bing Search API 기본 사항에 대해 확실하게 이해할 수 있을 것입니다. 또한 **Callback Handler, 사용법 및 봇 애플리케이션에서의 중요성**에 대해서도 알게 될 것입니다.

In [2]:
# Langchain을 사용하여 Azure OpenAI에 연결하는 데 필요한 ENV 변수를 설정
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

## Introduction to Callback Handlers


다음 설명은 Callback에 대한 Langchain 문서에서 직접 가져온 자료입니다.([링크](https://python.langchain.com/docs/modules/callbacks/)):

**Callbacks**:<br>
LangChain은 LLM 응용 프로그램의 다양한 단계에 연결할 수 있는 콜백 시스템을 제공합니다. 이는 로그, 모니터링, 스트리밍 및 기타 작업에 유용합니다. API를 통해 사용 가능한 Callback 인수를 사용하여 이러한 이벤트를 구독할 수 있습니다. 이 인수는 핸들러 개체의 목록입니다.

**Callback handlers**:<br>
CallbackHandler는 CallbackHandler 인터페이스를 구현하는 개체로, 구독할 수 있는 각 이벤트에 대한 메서드가 있습니다. CallbackManager는 이벤트가 트리거되면 각 핸들러에서 해당 메서드를 호출합니다.

--------------------
콜백에 대한 핸들러를 통합하여 응답이 스트리밍될 때, 이를 관찰하고 에이전트의 추론 과정에 대한 통찰력을 얻을 수 있습니다. 이는 봇의 응답을 사용자에게 스트리밍하고 응답을 기다리는 동안 진행 중인 프로세스에 대해 계속 정보를 제공하는 것을 목포료 할 때 매우 유용합니다.

저희의 커스텀 핸들러는 `common/callbacks.py` 에서 확인하실 수 있습니다.

In [6]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

# 콜백 핸들러와 함께 LLM 개체를 선언합니다.
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000)

# 혹은 응답이 스트리밍 되는 것을 보고 싶다면 아래 코드에서 코멘트를 해제합니다. 
# llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000, streaming=True, callback_manager=cb_manager)

## Creating a custom tool - Bing Search API tool

Langhain에는 이미 BingSearchAPIWrapper ([HERE](https://github.com/hwchase17/langchain/blob/master/langchain/utilities/bing_search.py))라는 미리 만들어진 툴이 있지만, 실행 기능 대신 결과 기능을 사용함으로써 좀 더 나은 방법으로 텍스트 결과뿐만 아니라 각 스니펫의 제목과 링크(소스)도 얻을 수 있습니다.

In [7]:
class MyBingSearch(BaseTool):
    """Tool for a Bing Search Wrapper"""
    
    name = "@bing"
    description = "useful when the questions includes the term: @bing.\n"

    k: int = 5
    
    def _run(self, query: str) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        return bing.results(query,num_results=self.k)
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

이제 맞춤형 도구와 맞춤형 프롬프트 `BING_PROMPT_PREFIX`를 사용하는 REACT Agent를 생성합니다.

In [8]:
tools = [MyBingSearch(k=5)]
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         agent_kwargs={'prefix':BING_PROMPT_PREFIX}, callback_manager=cb_manager, )

아래 질문들 중 일부를 시도해보시거나, 좋아할만 한 다른 질문들을 시도해보세요.

In [9]:
QUESTION = "Create a list with the main facts on What is happening with the oil supply in the world right now?"
# QUESTION = "How much is 50 USD in Euros and is it enough for an average hotel in Madrid?"
# QUESTION = "My son needs to build a pinewood car for a pinewood derbi, how do I build such a car?"
# QUESTION = "Who won the 2023 superbowl and who was the MVP?"
# QUESTION = "can I travel to Hawaii, Maui from Dallas, TX for 7 days with $7000 on the month of September, what are the best days to travel?"


# This complex question below needs gpt-4-32k (0613 version) in order to ensure a good answer. 
# ---------------
# QUESTION = """
# compare the number of job opennings (provide the exact number), the average salary within 15 miles of Dallas, TX, for these ocupations:

# - ADN Registerd Nurse 
# - Occupational therapist assistant
# - Dental Hygienist
# - Graphic Designer
# - Real Estate Agent


# Create a table with your findings. Place the sources on each cell.
# """

In [10]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

The user is asking for current information about the global oil supply. I will need to perform a web search to gather the most recent data and facts on this topic.
Action: @bing
Action Input: What is happening with the oil supply in the world right now?The user is asking for current information about the global oil supply. I will need to perform a web search to gather the most recent data and facts on this topic.
Action: @bing
Action Input: What is happening with the oil supply in the world right now?
The search results provide some information about the current state of the global oil supply. The United States is the world's top oil consumer, with China being the second<sup><a href="https://www.forbes.com/sites/rrapier/2023/08/18/the-us-still-leads-in-global-petroleum-production--consumption/" target="_blank">[1]</a></sup>. There are contrasting forecasts about oil demand growth, with the IEA lowering its forecast and OPEC maintaining a stronger growth forecast<sup><a href="https://ww

In [11]:
printmd(response)

Here are the main facts about the current state of the global oil supply:

1. The United States is the world's top oil consumer, with China being the second<sup><a href="https://www.forbes.com/sites/rrapier/2023/08/18/the-us-still-leads-in-global-petroleum-production--consumption/" target="_blank">[1]</a></sup>.
2. There are contrasting forecasts about oil demand growth, with the IEA lowering its forecast and OPEC maintaining a stronger growth forecast<sup><a href="https://www.reuters.com/markets/commodities/iea-lowers-2024-oil-demand-growth-forecast-2023-08-11/" target="_blank">[2]</a></sup>.
3. Oil prices have been on a streak of gains due to increased demand forecasts<sup><a href="https://www.bloomberg.com/news/articles/2023-08-11/latest-oil-market-news-and-analysis-for-august-11" target="_blank">[3]</a></sup>.
4. High oil prices are expected to continue<sup><a href="https://www.cnn.com/2022/06/03/energy/oil-prices-what-next/index.html" target="_blank">[4]</a></sup>.
5. There was a decline in production to 9.7 million BPD in May 2020, but it has since bounced back to 11.3 million BPD<sup><a href="https://www.forbes.com/sites/rrapier/2021/10/22/the-us-oil-supply-is-still-out-of-balance/" target="_blank">[5]</a></sup>.
6. The U.S. oil production hit an all-time high of just below 13 million barrels per day (BPD) before the Covid-19 pandemic, but demand collapsed as the pandemic unfolded<sup><a href="https://www.forbes.com/sites/rrapier/2022/03/11/what-is-holding-back-us-oil-production/" target="_blank">[6]</a></sup>.
7. OPEC+ is also considering the early 2022 oil market, with the possibility of more Covid cases in winter and a lower-demand “shoulder” season in spring<sup><a href="https://www.forbes.com/sites/daneberhart/2021/11/13/why-are-oil-prices-so-high-when-the-us-remains-one-of-the-worlds-largest-producers/" target="_blank">[7]</a></sup>.
8. Oil demand is expected to grow from 2022 to 2030, rising by just under 7 million b/d, but the rate of growth is expected to slow through the period to less than 0.5 million b/d a year by 2030<sup><a href="https://www.forbes.com/sites/woodmackenzie/2021/07/15/is-the-world-sleepwalking-into-an-oil-supply-crunch/" target="_blank">[8]</a></sup>.
9. An outage on the largest oil pipeline to the United States from Canada could affect inventories at a key U.S. storage hub<sup><a

## 특정 웹 사이트에 대한 QnA

스마트 봇이 특정 회사의 공개 웹 사이트에 대한 질문에 대답하기를 원하는 여러가지 활용 사례가 있으며, 우리가 취할 수 있는 두 가지 접근 방법은 다음과 같습니다 :

1. 정기적으로 실행되는 크롤러 스크립트를 만들어 웹사이트의 모든 페이지를 찾아내고 문서를 Azure Cognitive Search로 푸시합니다.
2. Bing이 이미 공개 웹사이트를 색인화하였을 가능성이 높기 때문에, 사이트를 직접 색인화하고 Bing 크롤러가 이미 수행한 작업을 복제하는 대신 해당 사이트를 대상으로 하는 Bing 검색을 활용할 수 있습니다.

다음은 특정 사이트와 관련된 몇 가지 예시 질문입니다. 다음을 살펴 보세요 :

In [12]:
# QUESTION = "information on how to kill wasps in homedepot.com"
# QUESTION = "in target.com, find how what's the price of a Nesspresso coffee machine and of a Keurig coffee machine"
QUESTION = "in microsoft.com, find out what is the latests news on quantum computing"
# QUESTION = "give me on a list the main points on the latest investor report from mondelezinternational.com"

In [13]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(3):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

The user is asking for the latest news on quantum computing from the Microsoft website. I will use the `site:` special operator to search for this information specifically on microsoft.com.
Action: @bing
Action Input: quantum computing news site:microsoft.comThe user is asking for the latest news on quantum computing from the Microsoft website. I will use the `site:` special operator to search for this information specifically on microsoft.com.
Action: @bing
Action Input: quantum computing news site:microsoft.com
The search results provide several pieces of news about quantum computing from Microsoft. The first snippet talks about Microsoft achieving the first milestone towards a quantum supercomputer. The second snippet discusses Azure Quantum demonstrating formerly elusive quantum phenomena. The third snippet mentions new data available for Microsoft's quantum machine on the Azure Quantum platform. The fourth snippet discusses new Microsoft breakthroughs that bring general-purpose qu

In [14]:
printmd(response)

Here are some of the latest news on quantum computing from Microsoft:

1. Microsoft has achieved the first milestone towards a quantum supercomputer<sup><a href="https://cloudblogs.microsoft.com/quantum/2023/06/21/microsoft-achieves-first-milestone-towards-a-quantum-supercomputer/" target="_blank">[1]</a></sup>.
2. Azure Quantum has demonstrated formerly elusive quantum phenomena<sup><a href="https://news.microsoft.com/source/features/innovation/azure-quantum-majorana-topological-qubit/" target="_blank">[2]</a></sup>.
3. New data is available for Microsoft's quantum machine on the Azure Quantum platform<sup><a href="https://cloudblogs.microsoft.com/quantum/2022/11/17/microsofts-quantum-machine-new-data-available-today/" target="_blank">[3]</a></sup>.
4. Microsoft has made breakthroughs that bring general-purpose quantum computing closer to reality<sup><a href="https://news.microsoft.com/features/new-microsoft-breakthroughs-general-purpose-quantum-computing-moves-closer-reality/" target="_blank">[4]</a></sup>.

Is there anything else you would like to know?

In [17]:
# Uncomment if you want to take a look at the custom bing search prompt (This is where the magic happens: a great system promp + GPT-4)
# printmd(agent_executor.agent.llm_chain.prompt.template)

# Summary

이 노트북에서는 콜백 핸들러에 대해 알아보고 LLM에서 응답을 스트리밍하는 방법에 대해 알아보았습니다. 또한 특정 검색 및 형식 지정 지침과 함께 영리한 프롬프트를 사용하여 Bing Chat 클론을 만드는 방법에 대해 알아봅니다.

결과는 당사를 대신하여 지능적인 웹 검색 및 연구 수행이 가능한 에이전트이며, 해당 URL 인용 및 링크와 함께 질문에 대한 답변을 제공합니다!